In [2]:
import pandas as pd

df= pd.read_csv('C:/Users/tjoeun/Downloads/미니프로젝트/데이터/finaldata.csv')

In [3]:
df['년월'] = df['년월'].astype(str)
# 년월로 groupby
df_time = df.groupby('년월')[['주택담보대출(연%)', '미국달러', '일본엔', 
                            '유로', '소비자물가지수','소비자물가지수(전월 대비 증감률)', '  KOSPI_종가', '  KOSDAQ_종가', '  무담보콜금리(1일)','KORIBOR(3개월)', 'CD(91일)', '국고채(1년)', '국고채(10년)', 
                            '면적당가격']].mean()

In [4]:
# 전처리
df_time.rename(columns={'주택담보대출(연%)':'주택담보대출금리',
                        '소비자물가지수(전월 대비 증감률)':'소비자물가지수증감률',
                        '  KOSPI_종가':'KOSPI_종가',
                        '  KOSDAQ_종가':'KOSDAQ_종가',
                        '  무담보콜금리(1일)':'무담보콜금리',
                        'KORIBOR(3개월)':'KORIBOR',
                        'CD(91일)':'CD',
                        '국고채(1년)':'국고채_1년',
                        '국고채(10년)':'국고채_10년'},
                        inplace=True)

In [5]:
import statsmodels.api as sm # 선영 회귀 모델
import statsmodels.formula.api as smf

In [6]:
model = smf.ols(formula = '''면적당가격 ~ 주택담보대출금리 + 미국달러 + 일본엔 + 유로
+ 소비자물가지수 + 소비자물가지수증감률 + KOSPI_종가 + KOSDAQ_종가
+ 무담보콜금리 + KORIBOR + CD + 국고채_1년 + 국고채_10년''', data=df_time).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  면적당가격   R-squared:                       0.885
Model:                            OLS   Adj. R-squared:                  0.877
Method:                 Least Squares   F-statistic:                     105.9
Date:                Mon, 27 Mar 2023   Prob (F-statistic):           1.06e-76
Time:                        12:07:47   Log-Likelihood:                -1158.5
No. Observations:                 193   AIC:                             2345.
Df Residuals:                     179   BIC:                             2391.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2944.2723    262.714    -11.207      0.000   -3462.688   -2425.857
주택담보대출금리      28.6341     38.501      0.744      0.458     -47.339     104.608
미국달러           0.4307      0.190      2.268      0.024       0.056       0.805
일본엔            4.0190      8.554      0.470      0.639     -12.861      20.899
유로             0.3242      0.127      2.549      0.012       0.073       0.575
소비자물가지수       19.8063      3.648      5.429      0.000      12.607      27.005
소비자물가지수증감률    21.0612     23.813      0.884      0.378     -25.929      68.052
KOSPI_종가       0.0629      0.066      0.958      0.340      -0.067       0.192
KOSDAQ_종가      0.9946      0.146      6.833      0.000       0.707       1.282
무담보콜금리      -173.0088     64.524     -2.681      0.008    -300.334     -45.684
KORIBOR      451.0353    193.137      2.335      0.021      69.917     832.154
CD          -198.5257    192.000     -1.034      0.303    -577.400     180.348
국고채_1년      -104.5263     67.306     -1.553      0.122    -237.342      28.290
국고채_10년       -2.1603     33.082     -0.065      0.948     -67.442      63.121
==============================================================================
Omnibus:                        6.610   Durbin-Watson:                   0.347
Prob(Omnibus):                  0.037   Jarque-Bera (JB):                9.334
Skew:                           0.175   Prob(JB):                      0.00940
Kurtosis:                       4.019   Cond. No.                     1.08e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.08e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

##### 다중공선성

In [7]:
dfX = df_time[['주택담보대출금리', '미국달러', '일본엔', '유로', '소비자물가지수', '소비자물가지수증감률', 'KOSPI_종가',
       'KOSDAQ_종가', '무담보콜금리', 'KORIBOR', 'CD', '국고채_1년', '국고채_10년', '면적당가격']]

from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(dfX.values, i) for i in range(dfX.shape[1])]
vif['변수'] = dfX.columns
vif = vif.sort_values("VIF").reset_index(drop=True)
vif

,VIF,변수
0,1.621449,소비자물가지수증감률
1,41.196639,면적당가격
2,142.767517,KOSDAQ_종가
3,164.267605,일본엔
4,251.380273,국고채_10년
5,320.666848,KOSPI_종가
6,500.527062,주택담보대출금리
7,501.411370,무담보콜금리
8,560.601041,유로
9,657.887252,국고채_1년


In [8]:
df = df.rename(columns={'면적당가격':'면적당가격',
                        '구':'구',
                        '동':'동',
                        '년월':'월',
                        '주택담보대출(연%)':'주택담보대출',
                        '미국달러':'달러',
                        '일본엔':'엔화',
                        '유로':'유로',
                        '소비자물가지수':'물가지수',
                        '  KOSPI_종가':'코스피',
                        '  KOSDAQ_종가':'코스닥',
                        '  무담보콜금리(1일)':'무담보금리',
                        'KORIBOR(3개월)':'코리보',
                        'CD(91일)':'cd',
                        '국고채(1년)':'국고채1년',
                        '국고채(10년)':'국고채10년'})

##### 독립변수 제거

In [9]:
#다중공선성 높은 순으로 제거
# KORIBOR 제거
model = smf.ols(formula = '''면적당가격 ~ 주택담보대출금리 + 미국달러 + 일본엔 + 유로
+ 소비자물가지수 + 소비자물가지수증감률 + KOSPI_종가 + KOSDAQ_종가
+ 무담보콜금리 + CD + 국고채_1년 + 국고채_10년''', data=df_time).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  면적당가격   R-squared:                       0.881
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                     111.5
Date:                Mon, 27 Mar 2023   Prob (F-statistic):           1.41e-76
Time:                        12:07:47   Log-Likelihood:                -1161.4
No. Observations:                 193   AIC:                             2349.
Df Residuals:                     180   BIC:                             2391.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3041.0928    262.612    -11.580      0.000   -3559.287   -2522.899
주택담보대출금리      47.5632     38.101      1.248      0.214     -27.618     122.744
미국달러           0.3720      0.190      1.953      0.052      -0.004       0.748
일본엔            4.6871      8.654      0.542      0.589     -12.390      21.764
유로             0.3383      0.129      2.631      0.009       0.085       0.592
소비자물가지수       20.6566      3.675      5.621      0.000      13.406      27.908
소비자물가지수증감률    13.5658     23.886      0.568      0.571     -33.567      60.698
KOSPI_종가       0.0651      0.066      0.980      0.328      -0.066       0.196
KOSDAQ_종가      0.9993      0.147      6.782      0.000       0.709       1.290
무담보콜금리      -164.1701     65.205     -2.518      0.013    -292.834     -35.507
CD           215.2500     74.871      2.875      0.005      67.512     362.988
국고채_1년       -86.1631     67.667     -1.273      0.205    -219.687      47.360
국고채_10년       -4.4751     33.474     -0.134      0.894     -70.527      61.577
==============================================================================
Omnibus:                        7.194   Durbin-Watson:                   0.348
Prob(Omnibus):                  0.027   Jarque-Bera (JB):               10.593
Skew:                           0.184   Prob(JB):                      0.00501
Kurtosis:                       4.087   Cond. No.                     1.02e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.02e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
# cd 제거
model = smf.ols(formula = '''면적당가격 ~ 주택담보대출금리 + 미국달러 + 일본엔 + 유로
+ 소비자물가지수 + KOSPI_종가 + KOSDAQ_종가
+ 무담보콜금리+ 국고채_1년 + 국고채_10년''', data=df_time).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  면적당가격   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.869
Method:                 Least Squares   F-statistic:                     128.5
Date:                Mon, 27 Mar 2023   Prob (F-statistic):           6.20e-77
Time:                        12:07:48   Log-Likelihood:                -1165.7
No. Observations:                 193   AIC:                             2353.
Df Residuals:                     182   BIC:                             2389.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3234.2313    253.145    -12.776      0.000   -3733.708   -2734.755
주택담보대출금리     103.2942     33.319      3.100      0.002      37.553     169.036
미국달러           0.5269      0.185      2.855      0.005       0.163       0.891
일본엔           -0.9383      8.524     -0.110      0.912     -17.758      15.881
유로             0.3888      0.129      3.007      0.003       0.134       0.644
소비자물가지수       20.3174      3.682      5.518      0.000      13.053      27.582
KOSPI_종가       0.0816      0.067      1.216      0.226      -0.051       0.214
KOSDAQ_종가      0.9211      0.145      6.342      0.000       0.635       1.208
무담보콜금리       -21.4988     42.899     -0.501      0.617    -106.143      63.145
국고채_1년       -20.2318     62.690     -0.323      0.747    -143.925     103.461
국고채_10년      -38.1974     30.841     -1.239      0.217     -99.049      22.654
==============================================================================
Omnibus:                        3.868   Durbin-Watson:                   0.300
Prob(Omnibus):                  0.145   Jarque-Bera (JB):                4.254
Skew:                           0.138   Prob(JB):                        0.119
Kurtosis:                       3.673   Cond. No.                     9.57e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.57e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
#### 더 안 빼도 괜찮나?
#### 점수가 안 좋다고해서 다중공선성 높은 요소들을 다 넣고 모델링 해도 괜찮나?

##### 전체데이터로 모델 학습 & 테스트

In [12]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [15]:
# LinearRegression
from sklearn.linear_model import LinearRegression

X = df[['주택담보대출', '달러','엔화','유로','물가지수','코스피','코스닥','무담보금리','코리보','cd','국고채1년','국고채10년']]
y = df['면적당가격']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10)
model = LinearRegression().fit(X_train, y_train)

print("학습용:",model.score(X_train, y_train))
print("검증용:",model.score(X_test, y_test))

학습용: 0.2446850308398295
검증용: 0.24239954197950364


In [16]:
# DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
model = DecisionTreeRegressor().fit(X_train, y_train)

print("학습용 : ", model.score(X_train, y_train))
print("검증용 : ", model.score(X_test, y_test))

학습용 :  0.29629876772623276
검증용 :  0.29528587741592605


In [17]:
# RandomForestRegressor 학습
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
model = RandomForestRegressor(n_jobs = -1).fit(X_train, y_train)

print("학습용 : ", model.score(X_train, y_train))
print("검증용 : ", model.score(X_test, y_test))

학습용 :  0.2962966763476801
검증용 :  0.29528257325199525


#### 구별 분석

In [40]:
df = pd.read_csv('C:/Users/tjoeun/Downloads/미니프로젝트/데이터/finaldata.csv')
a = input()
df_region = df[df['구']==a]
print(a)

중랑구


In [41]:
# 전처리
df_region.rename(columns={'주택담보대출(연%)':'주택담보대출',
                        '소비자물가지수(전월 대비 증감률)':'소비자물가지수증감률',
                        '  KOSPI_종가':'KOSPI_종가',
                        '  KOSDAQ_종가':'KOSDAQ_종가',
                        '  무담보콜금리(1일)':'무담보콜금리',
                        'KORIBOR(3개월)':'KORIBOR',
                        'CD(91일)':'CD',
                        '국고채(1년)':'국고채_1년',
                        '국고채(10년)':'국고채_10년'},
                        inplace=True)

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_12844\3485181508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_region.rename(columns={'주택담보대출(연%)':'주택담보대출',


In [42]:
df_time = df_region.groupby('년월')[['전용면적', '층', '건축년도', '계약년', '계약월',
       '주택담보대출', '미국달러', '일본엔', '유로',
       'KOSPI_종가', 'KOSDAQ_종가', '무담보콜금리', 'KORIBOR', 'CD',
       '국고채_1년', '국고채_10년','면적당가격','년월']].mean()

In [43]:
X = df_time[['전용면적', '층', '건축년도','KOSPI_종가', '미국달러', '일본엔', '유로', 'KOSDAQ_종가','계약월']]
y = df_time['면적당가격']

df_time[df_time['년월'] == 202301]['면적당가격'].mean()

827.8349638964355

In [45]:
# LinearRegression 학습
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
model = LinearRegression().fit(X_train, y_train)

print("학습용 : ", model.score(X_train, y_train))
print("검증용 : ", model.score(X_test, y_test))

# LinearRegression 예측
# 실제 2023-01 데이터 입력 후 실제 가격과 예측값 비교하기
predict = model.predict([[85.5, 9, 2020,2425.08, 1228.7, 9.4222, 1332.83, 740.49,5]])
print('예측값 : ', predict)

학습용 :  0.8898795575107963
검증용 :  0.8416635479358058
예측값 :  [719.50648707]


c:\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [ ]:
# DecisionTreeRegressor 학습
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

# 할 때마다 다른 값이 나오기때문에, 1000회의 평균값으로 구하기
predict2 = []
for i in range(0,1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
    model = DecisionTreeRegressor().fit(X_train, y_train)
    predict = model.predict([[85.5, 9, 2020,2425.08, 1228.7, 9.4222, 1332.83, 740.49,5]])
    predict2.append(predict[0])

print("학습용 : ", model.score(X_train, y_train))
print("검증용 : ", model.score(X_test, y_test))
print('예측값 : ', predict)

In [48]:
# RandomForestRegressor 학습
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

predict3 = []
for i in range(0,1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
    model = RandomForestRegressor(n_jobs = -1).fit(X_train, y_train)
    predict = model.predict([[85.5, 9, 2020,2425.08, 1228.7, 9.4222, 1332.83, 740.49,5]])
    predict3.append(predict[0])
    
print("학습용 : ", model.score(X_train, y_train))
print("검증용 : ", model.score(X_test, y_test))
print("예측값 : ",sum(predict3)/len(predict3))

c:\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegress

학습용 :  0.9847925374771764
검증용 :  0.9147540375293065
예측값 :  646.0235908827137


c:\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [38]:
# 실제 값과 비교
# 2023-01 실거래가 : 827.834

In [49]:
# 2024-01 예측
# LinearRegression 학습
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
model = LinearRegression().fit(X_train, y_train)

print("학습용 : ", model.score(X_train, y_train))
print("검증용 : ", model.score(X_test, y_test))

# LinearRegression 예측
# 실제 2023-01 데이터 입력 후 실제 가격과 예측값 비교하기
# [['전용면적', '층', '건축년도','KOSPI_종가', '미국달러', '일본엔', '유로', 'KOSDAQ_종가','계약월']]
predict = model.predict([[85.5, 9, 2020, 2380, 1301, 9.92, 1397.85, 730, 1]])
print('예측값 : ', predict)

학습용 :  0.8898795575107963
검증용 :  0.8416635479358058
예측값 :  [761.74160349]


c:\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
